In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import gmaps

# Import the datetime module from the datetime library.
from datetime import datetime

# Import the requests library.
import requests

# Import the API keys.
from config import weather_api_key
from config import g_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [2]:
# Create 2000 random latitudes and longitudes combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Get the nearest city from the coordinates.
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

762

In [33]:
# Create an empty list to hold the weather data.
city_data = []

# Loop through the cities list to build an api call for each city
for i in range(len(cities)):
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]

In [34]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Description": city_description})

# If an error is experienced, skip the city.
    except:
        pass

In [35]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,Mataura,-46.1927,168.8643,35.47,99,100,0.94,NZ,overcast clouds
1,Pevek,69.7008,170.3133,41.58,76,24,4.74,RU,few clouds
2,Hanno,35.8519,139.3181,77.67,96,90,10.36,JP,heavy intensity rain
3,Aksha,50.2814,113.2867,53.46,94,5,4.36,RU,clear sky
4,Petropavlovsk-Kamchatskiy,53.0452,158.6483,45.19,100,75,4.47,RU,broken clouds
...,...,...,...,...,...,...,...,...,...
700,Londoko,49.0333,131.9833,58.60,96,100,5.46,RU,overcast clouds
701,Karaton,-6.3424,106.1045,78.96,93,14,0.94,ID,few clouds
702,Kendari,-3.9450,122.4989,73.49,89,100,1.79,ID,overcast clouds
703,Mildura,-34.2000,142.1500,50.07,89,78,2.98,AU,broken clouds


In [36]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")